In [8]:
import gym
env = gym.make('FrozenLake-v0', is_slippery=False)


In [9]:
import time
env.s = env.reset()
env.s = env.observation_space.sample()
env.render()

# Movement is slippery, meaning that agent won't always move in the directed stated
# Actions: 0, 1, 2, 3 for left, down, right, up, respectively



SFFF
FHFH
FFFH
HFFG


In [10]:
env.observation_space.n # Total number of states

16

In [11]:
env.action_space.n # Total number of actions

4

In [ ]:
env.reward_range # Reward range: 0 for not reaching goal, 1 for reaching goal

(0, 1)

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, delay):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(delay)

In [ ]:
# Let's see what happens if we try to brute force,
env.reset()
epochs = 0
penalties, reward = 0, 0
frames = [] # for animation
reward = 0
done = False
rewards = []

while reward != 1:
    action = env.action_space.sample() # Chosing random action
    state, reward, done, info = env.step(action) # Extracting info
    rewards.append(reward)
    if state == 5 or state == 7 or state == 11 or state == 12:
        penalties += 1
        env.s = env.reset()
    #print(reward)
        
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1

print_frames(frames, 0.05)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 350
State: 15
Action: 2
Reward: 1.0
Timesteps taken: 350
Penalties incurred: 47


In [ ]:
# ** TRAINING THE AGENT **
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n]) # Initialize the Q-table to a 16 x 4 matrix of zeros
q_table

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
import random
from IPython.display import clear_output

# Hyper parameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []
randoms = 0

for i in range(1, 10000):
    state = env.reset()
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            randoms += 1
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) #Exploit learned values
        
        next_state, reward, done, info = env.step(action)
        if next_state in [5, 7, 10, 11]:
            reward -= 1
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha)*old_value + alpha*(reward + gamma*next_max)
        q_table[state, action] = new_value
        
        if reward == 0:
            penalties += 1
        
        state = next_state
        epochs += 1
        
        if i % 100 == 0:
            clear_output(wait=True)
            print('Episode: {}'.format(i))
    
print("Training finished. \n")
            
    

Episode: 9900
Training finished. 



In [ ]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.1       , -0.1       , -0.1       ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.98669721, -0.9835768 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [ ]:
env.s = env.reset()
state = env.s
epochs = 0
penalties, reward = 0, 0
frames = [] # for animation
done = False 

while reward != 1:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == 0:
        penalties += 1
        
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
print_frames(frames, 0.5)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))